In [1]:
import pickle
from string import punctuation

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
import json
import dask.bag as db

In [10]:
# Load the data
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

import dask.bag as db
import json

docs = db.read_text('../Data/arxiv-metadata-oai-snapshot.json').map(json.loads)

In [11]:
# Trimming the data
get_latest_version = lambda x: x['versions'][-1]['created']


# get only necessary fields of the metadata file
trim = lambda x: {'id': x['id'],
                  'versions': x['versions'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'],}

# filter for papers published before 2022
columns = ['id','category','abstract','versions']
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) <2022)
           .map(trim).
           compute())

# convert to pandas
docs_df = pd.DataFrame(docs_df)

In [14]:
#save trimmed dataset for later use so we can skip the dataset trimming later:
docs_df.to_csv("../Data/trimmed_arxiv_docs3.csv", index=False)

In [15]:
# Read docs from the file
df = pd.read_csv('../Data/trimmed_arxiv_docs3.csv')

In [16]:
categories = list(df['category'].unique())

In [17]:
# Split categories
def splitCategory(cat):
    tmp = cat.replace('[','').replace(']','')
    c = tmp.split(', ')
    c = [w.replace("'","") for w in c]
    return c

In [18]:
# Find main category
def findMainCat(categories):
    new_cat = list(dict.fromkeys([cat.split(".")[0] for cat in categories]))
    return new_cat

In [19]:
# Getting rid of duplicates
df.drop_duplicates(['abstract',], inplace=True)
df['abstract'].describe(include='all')

count                                               2038555
unique                                              2038555
top         A fully differential calculation in perturba...
freq                                                      1
Name: abstract, dtype: object

NLP Processing

In [21]:
# Add the categories
df['list_categories'] = df.category.apply(splitCategory)
df['main_categories'] = df.list_categories.apply(findMainCat)

In [22]:
# Create CS and Random dataframes
bool_series = ["cs" in cat[0] for cat in df.main_categories]
df_cs = df[bool_series]

In [23]:
# Non-CS df
bool_series = ["cs" not in cat for cat in df.main_categories]
df_random = df[bool_series]

In [25]:
def get_year(version):
    return int(version.split(': ')[2].split(' ')[3])

In [26]:
import datetime
def get_month(version):
    month_name = version.split(': ')[2].split(' ')[2]
    datetime_object = datetime.datetime.strptime(month_name, "%b")
    month_number = datetime_object.month
    return month_number

In [27]:
def get_quarter(version):
    year = get_year(version)
    month = get_month(version)
    date = int(year)*12+int(month)
    return date

In [28]:
df_cs['year'] = df_cs.versions.apply(get_year)
df_random['year'] = df_random.versions.apply(get_year)
df_cs['quarter'] = df_cs.versions.apply(get_quarter)
df_random['quarter'] = df_random.versions.apply(get_quarter)

/var/folders/9k/9qw6yrzx6493tcr085_zrwj40000gn/T/ipykernel_58356/3872019404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cs['year'] = df_cs.versions.apply(get_year)
/var/folders/9k/9qw6yrzx6493tcr085_zrwj40000gn/T/ipykernel_58356/3872019404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_random['year'] = df_random.versions.apply(get_year)
/var/folders/9k/9qw6yrzx6493tcr085_zrwj40000gn/T/ipykernel_58356/3872019404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [31]:
# Exclude 2022
bool_series = [year<2022 for year in df_cs.year]
df_cs = df_cs[bool_series]

In [32]:
# Exclude 2022
bool_series = [year<2022 for year in df_random.year]
df_random = df_random[bool_series]

In [ ]:
# Reduce the size of the data
df_cs = df_cs[].sample(100000, random_state=42)
df_random = df_random.sample(100000, random_state=42)

In [38]:
df_cs.loc[2]

id                                                          704.0003
authors                                                  Hongjun Pan
title              The evolution of the Earth-Moon system based o...
versions           [{'version': 'v1', 'created': 'Sun, 1 Apr 2007...
category                                          ['physics.gen-ph']
abstract             The evolution of Earth-Moon system is descri...
list_categories                                     [physics.gen-ph]
main_categories                                            [physics]
year                                                            2007
quarter                                                        24088
Name: 2, dtype: object